In [1]:
# Стандартні бібліотеки
import os
import logging

# Зовнішні бібліотеки
import numpy as np
import PIL
import tensorflow as tf
from keras.models import load_model
from keras.preprocessing.image import  img_to_array

In [2]:
channels = 1
cropped_width = 498
cropped_height = 300
upscale_factor = 3

input_width = cropped_width // upscale_factor
input_height = cropped_height  // upscale_factor

TEST_FOLDER_PATH  = 'D:\\KPI\\Bachelor_thesis\\code\\data\\images\\test'
TRAIN_FOLDER_PATH = 'D:\\KPI\\Bachelor_thesis\\code\\data\\images\\train'
VAL_FOLDER_PATH   = 'D:\\KPI\\Bachelor_thesis\\code\\data\\images\\val'

RAW_TEST_FOLDER_PATH  = 'D:\\KPI\\Bachelor_thesis\\code\\data\\raw_images\\test'
RAW_TRAIN_FOLDER_PATH = 'D:\\KPI\\Bachelor_thesis\\code\\data\\raw_images\\train'
RAW_VAL_FOLDER_PATH   = 'D:\\KPI\\Bachelor_thesis\\code\\data\\raw_images\\val'

In [3]:
model = load_model('D:\\KPI\\model.h5')

In [4]:
total_bicubic_psnr = 0.0      # Variable for total average LR-HR PSNR.
total_test_psnr = 0.0         # Variable for total average ER-HR PSNR.

valid_path = TEST_FOLDER_PATH                                           # Path of the folder containing  
                                                                        # images for validation.
    
valid_img_paths = sorted([os.path.join(valid_path,img_name)             # List comprehension to get the
                        for img_name in os.listdir(valid_path)])[:10_000]        # full path of the images.

psnr_values = {}  # Dictionary to store the PSNR values for each image

for index, _ in enumerate(valid_img_paths):
    
    """ -------- VISUAL ASSESSMENT -------- """
    
    HR_image = PIL.Image.open(_)
    
    LR_image = HR_image.resize((HR_image.size[0] // upscale_factor,   # Donwscaling to LR occurs here.
                                HR_image.size[1] // upscale_factor),
                               PIL.Image.BICUBIC)
    
    ycbcr = LR_image.convert("YCbCr")       # Here, we converted the image to YCbCr because we
    y, cb, cr = ycbcr.split()               # need the Y channel for our model. We did not convert
    y = img_to_array(y)                     # it to YUV because we can't in PIL and Y in YCbCr is
    y = y.astype("float32") / 255.0         # the same as in YUV.
                                            
    model_input = y.reshape(1, y.shape[0], y.shape[1], y.shape[2])      # (Batch, Width, Height, Channels)
    
    output = model.predict(model_input)
    output = output[0]                                            # (Width, Height, Channels)
    output *= 255.0
    output = output.clip(0, 255)
    output = output.reshape((output.shape[0], output.shape[1]))   # (Width, Height)
    output = PIL.Image.fromarray(np.uint8(output))
    output = output.resize(HR_image.size, PIL.Image.Resampling.NEAREST)   # This step is necessary to
                                                                          # fill missing pixels from
                                                                          # the prediction process.
            
    cb = cb.resize(output.size, PIL.Image.Resampling.BICUBIC)          
    cr = cr.resize(output.size, PIL.Image.Resampling.BICUBIC)
    
    ER_image = PIL.Image.merge("YCbCr", (output, cb, cr))
    ER_image = ER_image.convert("RGB")
     
    LR_image = LR_image.resize(ER_image.size, PIL.Image.Resampling.BICUBIC)  # Resizing LR for plotting.
                                                                             
    """ -------- ACCURACY CALCULATION --------"""
    
    LR_arr = img_to_array(LR_image)
    HR_arr = img_to_array(HR_image)
    ER_arr = img_to_array(ER_image)
    
    bicubic_psnr = tf.image.psnr(LR_arr, HR_arr, max_val=255)
    test_psnr = tf.image.psnr(ER_arr, HR_arr, max_val=255)

    total_bicubic_psnr += bicubic_psnr
    total_test_psnr += test_psnr

    psnr_values[_] = (bicubic_psnr, test_psnr, bicubic_psnr - test_psnr)  # Store the PSNR values and their difference


1/1 [==============================] - 1s 834ms/step


In [5]:
# Get the top 10 images with the highest PSNR difference
top_10_images = sorted(psnr_values, key=lambda x: psnr_values[x][1] - psnr_values[x][0], reverse=True)[:10]

print("Top 10 images with the most improvement:")
for img in top_10_images:
    print(f"{img}: LR-HR PSNR = {psnr_values[img][0]:.3f},  ER-HR PSNR = {psnr_values[img][1]:.3f},  Diff = {psnr_values[img][1] - psnr_values[img][0]:.3f}")


Top 10 images with the most improvement:
D:\KPI\Bachelor_thesis\code\data\images\test\ILSVRC2012_test_00001830.JPEG: LR-HR PSNR = 27.639,  ER-HR PSNR = 33.194,  Diff = 5.555
D:\KPI\Bachelor_thesis\code\data\images\test\ILSVRC2012_test_00004047.JPEG: LR-HR PSNR = 22.456,  ER-HR PSNR = 27.780,  Diff = 5.324
D:\KPI\Bachelor_thesis\code\data\images\test\ILSVRC2012_test_00001324.JPEG: LR-HR PSNR = 23.856,  ER-HR PSNR = 29.123,  Diff = 5.266
D:\KPI\Bachelor_thesis\code\data\images\test\ILSVRC2012_test_00014925.JPEG: LR-HR PSNR = 31.199,  ER-HR PSNR = 36.161,  Diff = 4.962
D:\KPI\Bachelor_thesis\code\data\images\test\ILSVRC2012_test_00011232.JPEG: LR-HR PSNR = 22.390,  ER-HR PSNR = 27.274,  Diff = 4.884
D:\KPI\Bachelor_thesis\code\data\images\test\ILSVRC2012_test_00011379.JPEG: LR-HR PSNR = 22.023,  ER-HR PSNR = 26.394,  Diff = 4.372
D:\KPI\Bachelor_thesis\code\data\images\test\ILSVRC2012_test_00007490.JPEG: LR-HR PSNR = 26.676,  ER-HR PSNR = 31.014,  Diff = 4.338
D:\KPI\Bachelor_thesis\code\

In [6]:
print(f"Середній PSNR між LR та HR зображеннями = {round(float((total_bicubic_psnr / len(valid_img_paths))), 3)}")
print(f"Середній PSNR між ER та HR зображеннями = {round(float((total_test_psnr / len(valid_img_paths))), 3)}")

Середній PSNR між LR та HR зображеннями = 25.704
Середній PSNR між ER та HR зображеннями = 26.351


In [7]:
# Сортуємо всі зображення за різницею PSNR
sorted_images = sorted(psnr_values, key=lambda x: psnr_values[x][1] - psnr_values[x][0], reverse=True)

# Відкриваємо файл для запису
with open('testing_model_no_draw.txt', 'w') as f:
    # Записуємо результати для кожного зображення
    for img in sorted_images:
        f.write(f"{img}: LR-HR PSNR = {psnr_values[img][0]:.6f},  ER-HR PSNR = {psnr_values[img][1]:.6f},  Diff = {psnr_values[img][1] - psnr_values[img][0]:.6f}\n")
